In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

#variable
SUMMARY = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/LR_SummaryTest_Table.tsv'
TSS = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/Merged_Causality_TSS_TSS.tab'
ANNOT = '/storage/resources/dbase/human/hg19/gencode_gene_annotations_hg19.csv'
sep='"\t"'

def GetDistTss(x):
    if x["gene.strand"] == "+":
        return x["str.start"] - x["gene.start"]
    elif x["gene.strand"] == "-":
        return -1*(x["str.start"] - x["gene.stop"])
    else:
        return float("nan")

def GetDistTes(x):
    if x["gene.strand"] == "+":
        return x["str.start"] - x["gene.stop"]
    elif x["gene.strand"] == "-":
        return -1*(x["str.start"] - x["gene.start"])
    else:
        return float("nan")
    

command = "cat %s |awk -F'\t' '{print $1%s$2%s$75%s$4}' > all_analyzed_strs.tab"%(SUMMARY, sep,sep,sep)
value = os.system(command)

In [ ]:

summary.merge(annot, on='gene', how='left')

In [4]:
#within dist_tss=1500bps of TSS or TES  (TS)
dist_ts = 1500

summary=pd.read_csv(SUMMARY, sep='\t')
summary['str.atrt'] = summary['str.start'].astype(int)
summary['locus'] = summary.apply(lambda x: x["chrom"]+":"+str(x["str.start"]), 1)

annot = pd.read_csv(ANNOT)[['gene.id','gene.start','gene.stop','gene.strand']]
annot['gene'] = annot['gene.id']
del annot['gene.id']
summary = summary.merge(annot, on='gene', how='left')
summary['dist.to.tss'] = summary.apply(lambda x: GetDistTss(x),1)
summary['dist.to.tes'] = summary.apply(lambda x: GetDistTes(x),1)

ts_data=pd.read_csv(TSS, sep='\t')
ts_data = ts_data.dropna(subset=['best.str.start'])
ts_data['str.start'] = ts_data['best.str.start'].astype(int)
ts_data['locus'] = ts_data.apply(lambda x: x['chrom']+':'+ x['str.start'].astype(str),1)
del ts_data['best.str.start']




ValueError: Cannot convert non-finite values (NA or inf) to integer

In [21]:
#tes
tes = annot[['gene.chr', 'gene.stop', 'attes', 'gene.id']].copy()
tes.columns=['chrom','start', 'stop','gene']
tes.to_csv('dist_tes.tab', index=None, sep='\t')
command = "bedtools intersect -a dist_tes.tab -b all_analyzed_strs.tab -wo > intersect_tes.tab"
value = os.system(command)

inter_tes = pd.read_csv('intersect_tes.tab', sep='\t', header=None)
inter_tes = inter_tes.loc[inter_tes[3]==inter_tes[7]][[4,5,6,7]]
inter_tes.columns=['chrom','str.start', 'str.end','gene']

estrs=pd.read_csv(SUMMARY, sep='\t'); print (len(set(list(estrs['chrom']+estrs['str.id']))), '\t',len(set(list(estrs['gene']))))
estrs=estrs.loc[estrs['E.tissues']>=1] ; print (len(set(list(estrs['chrom']+estrs['str.id']))), '\t',len(set(list(estrs['gene']))))
print(estrs.shape, inter_tes.shape)

I = inter_tes.merge(estrs, on=['chrom','str.start', 'str.end','gene'], how='inner')
print (len(set(list(I['chrom']+I['str.id']))), '\t',len(set(list(I['gene']))))
I.shape

125758 	 17517
20192 	 11869
(25619, 77) (1714, 4)
278 	 274


(278, 77)

In [ ]:
#TSS =========================================================
         TSS  	non TSS 	 Total STR
estrs 	 303 	19889   	 20192
nonestr	 1376	104190  	105566
Total	 1679	124079  	125758

Chi-square with Yates correction
  Chi squared equals 4.852 with 1 degrees of freedom. 
  The two-tailed P value equals 0.0276

#TES =========================================================
        TES  	 nonTES 	 Total STR
estrs 	 278	 19914  	20192
non-est	 1436	 104130 	105566
Total	 1714	 124044 	125758

Chi-square with Yates correction
  Chi squared equals 0.023 with 1 degrees of freedom. 
  The two-tailed P value equals 0.8791
